# 1. Install Package

In [ ]:
pip install boto3
pip install watchdog


# 2. Upload Image to S3

In [ ]:
import boto3
import os
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [ ]:
# Initialize the S3 client
session = boto3.Session(profile_name='profile1')
s3 = session.client('s3')

# Define the S3 bucket name and the local folder path
bucket_name = 'image-storage--testing'
local_folder = r'C:\Users\chuth\OneDrive\ImageTest'
# Define the path to the LINK.txt file
link_file = os.path.join(local_folder, 'LINK.txt')

# Function to upload an image to S3
def upload_image_to_s3(image_file, bucket_name):
    # Generate the key name for the image
    key_name = os.path.join('images', os.path.basename(image_file))
    # Upload the image to S3
    try:
        with open(image_file, 'rb') as image_data:
            s3.upload_fileobj(image_data, bucket_name, key_name)
        print(f"Image '{image_file}' uploaded successfully to S3!")
    except FileNotFoundError:
        print(f"Error uploading image '{image_file}' to S3: No such file or directory")
    except Exception as e:
        print(f"Error uploading image '{image_file}' to S3:", e)

In [ ]:
# Function to get the URL link of an object in S3
def get_s3_object_url(bucket, key):
    return f"https://{bucket}.s3.amazonaws.com/{key}"
  
# Event handler class to monitor changes in the local folder
class ImageUploadHandler(FileSystemEventHandler):
    def __init__(self):
        self.uploaded_files = set()

    def on_created(self, event):
        if event.is_directory:
            return
        filename = event.src_path
        if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')) and filename not in self.uploaded_files:
            time.sleep(1)
            upload_image_to_s3(filename, bucket_name)
            time.sleep(2)
            url = get_s3_object_url(bucket_name, f'images/{os.path.basename(filename)}')
            with open(link_file, 'a') as f:
                f.write(url + '\n')
            self.uploaded_files.add(filename)

# Function to start monitoring the local folder for image uploads
def start_monitoring():
    event_handler = ImageUploadHandler()
    observer = Observer()
    observer.schedule(event_handler, local_folder, recursive=False)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Start monitoring the local folder
start_monitoring()
